
# Pretrained Model
## Imports

In [29]:
import os
from tensorflow import keras
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import pydot


ModuleNotFoundError: No module named 'pydot'

In [2]:
# The directory where we will store our smaller dataset/
base_dir = "data/lab_01_split"

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


Homer = 'homer_simpson'
Bart = 'bart_simpson'
Burns = 'charles_montgomery_burns'
Krusty = 'krusty_the_clown'
Lisa = 'lisa_simpson'
Milhouse = 'milhouse_van_houten'
Marge = 'marge_simpson'
Moe = 'moe_szyslak'
Ned = 'ned_flanders'
Principal = 'principal_skinner'


# Directory with our training images
train_homer = os.path.join(train_dir, Homer)
train_bart = os.path.join(train_dir, Bart)
train_burns = os.path.join(train_dir, Burns)
train_krusty = os.path.join(train_dir, Krusty)
train_lisa = os.path.join(train_dir, Lisa)
train_milhouse = os.path.join(train_dir, Milhouse)
train_marge = os.path.join(train_dir, Marge)
train_more = os.path.join(train_dir, Moe)
train_ned = os.path.join(train_dir, Ned)
train_principal = os.path.join(train_dir, Principal)

# Directory with our valiation images
val_homer = os.path.join(validation_dir, Homer)
val_bart = os.path.join(validation_dir, Bart)
val_burns = os.path.join(validation_dir, Burns)
val_krusty = os.path.join(validation_dir, Krusty)
val_lisa = os.path.join(validation_dir, Lisa)
val_milhouse = os.path.join(validation_dir, Milhouse)
val_marge = os.path.join(validation_dir, Marge)
val_more = os.path.join(validation_dir, Moe)
val_ned = os.path.join(validation_dir, Ned)
val_principal = os.path.join(validation_dir, Principal)

# Directory with our test images
test_homer = os.path.join(test_dir, Homer)
test_bart = os.path.join(test_dir, Bart)
test_burns = os.path.join(test_dir, Burns)
test_krusty = os.path.join(test_dir, Krusty)
test_lisa = os.path.join(test_dir, Lisa)
test_milhouse = os.path.join(test_dir, Milhouse)
test_marge = os.path.join(test_dir, Marge)
test_more = os.path.join(test_dir, Moe)
test_ned = os.path.join(test_dir, Ned)
test_principal = os.path.join(test_dir, Principal)


In [3]:
print('total training Homer images:', len(os.listdir(train_homer)))
print('total validation Homer images:', len(os.listdir(val_homer)))
print('total test Homer images:', len(os.listdir(test_homer)))

total training Homer images: 1572
total validation Homer images: 337
total test Homer images: 337


# 2. Preprocessing


In [4]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

In [5]:
train_generator = train_datagen.flow_from_directory(
        train_dir, # target directory
        class_mode='categorical')

Found 9663 images belonging to 10 classes.


In [6]:
validation_generator = validation_datagen.flow_from_directory(
        validation_dir, # validation directory
        class_mode='categorical')

Found 2072 images belonging to 10 classes.


In [7]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (32, 256, 256, 3)
labels batch shape: (32, 10)


# 3. Building the Model



In [8]:
base_resnet50 = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape = (256, 256, 3))


In [9]:
# freeze the conv-net structure
base_resnet50.trainable = False



In [10]:


custom_model = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)),
        base_resnet50,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation='softmax'),
    ], name='global_model'
)


custom_model.summary()

Model: "global_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1310730   
Total params: 24,898,442
Trainable params: 1,310,730
Non-trainable params: 23,587,712
_________________________________________________________________


# 5. Training the model

In [11]:
earlystop = EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto')

csv_logger = CSVLogger('training.log',
                       separator=',',
                       append=False)

In [12]:
custom_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.8),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

Fitting the model

In [13]:
history = custom_model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=[earlystop, csv_logger]
)


Epoch 1/5
100/100 [==============================] - 428s 4s/step - loss: 10444.2490 - categorical_accuracy: 0.5663 - val_loss: 5394.9995 - val_categorical_accuracy: 0.6931
Epoch 2/5
100/100 [==============================] - 394s 4s/step - loss: 3279.5042 - categorical_accuracy: 0.7997 - val_loss: 4848.7256 - val_categorical_accuracy: 0.7506
Epoch 3/5
100/100 [==============================] - 398s 4s/step - loss: 3009.3203 - categorical_accuracy: 0.8459 - val_loss: 4607.5820 - val_categorical_accuracy: 0.7862
Epoch 4/5
100/100 [==============================] - 387s 4s/step - loss: 2254.8569 - categorical_accuracy: 0.8863 - val_loss: 6505.8574 - val_categorical_accuracy: 0.7575
Epoch 5/5
100/100 [==============================] - 389s 4s/step - loss: 2233.9563 - categorical_accuracy: 0.8947 - val_loss: 4389.9697 - val_categorical_accuracy: 0.8188


In [17]:
history.history

{'loss': [10444.2490234375,
  3279.504150390625,
  3009.3203125,
  2254.85693359375,
  2233.956298828125],
 'categorical_accuracy': [0.5662500262260437,
  0.7996875047683716,
  0.8458893299102783,
  0.8862500190734863,
  0.8946874737739563],
 'val_loss': [5394.99951171875,
  4848.7255859375,
  4607.58203125,
  6505.857421875,
  4389.9697265625],
 'val_categorical_accuracy': [0.6931250095367432,
  0.7506250143051147,
  0.7862499952316284,
  0.7574999928474426,
  0.8187500238418579]}

In [19]:
# Test Cell

test_gen= ImageDataGenerator().flow_from_directory(test_dir, class_mode='categorical')
results = custom_model.evaluate(test_gen)

Found 2076 images belonging to 10 classes.
65/65 [==============================] - 152s 2s/step - loss: 5128.8188 - categorical_accuracy: 0.8068


In [20]:
keras.utils.plot_model(custom_model,"Custom Model using Resnet")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [24]:
custom_model.save("pretrained_trained")

INFO:tensorflow:Assets written to: pretrained_trained/assets


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [27]:
sunflower_url = "https://static.wikia.nocookie.net/youtubepoop/images/e/eb/Homer-simpson_WOOHOO.jpg/revision/latest?cb=20130120090101"
sunflower_path = keras.utils.get_file('Homer', origin=sunflower_url)

img = keras.utils.load_img(
    sunflower_path, target_size=(256, 256)
)
img_array = keras.utils.img_to_array(img)
img_array = tensorflow.expand_dims(img_array, 0) # Create a batch

predictions = custom_model.predict(img_array)
score = tensorflow.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

139264/129088 [================================] - 0s 3us/step


NameError: name 'tensorflow' is not defined